In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

# import tensorflow as tf
# print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
!pip install tensorflow==2.0.0-beta0


# tf.enable_eager_execution()

!pip install -q tensorflow-datasets

     |████████████████████████████████| 87.9MB 114kB/s 
     |████████████████████████████████| 3.1MB 38.6MB/s 
     |████████████████████████████████| 501kB 30.7MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [3]:
import tensorflow as tf
print("check tensorflow version: ", tf.__version__)
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

check tensorflow version:  2.0.0-beta0


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
print ("example training sentences: ", training_sentences[403])
print ("example corresponding training labels: ", training_labels[403]) # 1 is positive review

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

example training sentences:  b'Karl Jr and his dad are now running an army on a remote island. They capture a trio of guys who stumble upon the island. Whom after a while fight back. (well the survivors) This one has non-stop blood, gore and carnage, which would have been good if any of it looked remotely real, or if the production didn\'t look like it was made with a weeks worth of saved up lunch money (I may be overexxagerating there. it was probably just a couple days worth). The horrendous dubbing didn\'t bother me as much and I suspect if I had been really drunk, some of it MIGHT have been slightly humorous....maybe. But as it is, at merely 78 minutes the movie still felt way too long by.. Oh I don\'t know... 78 minutes. Don\'t waste your time.<br /><br />My Grade: F <br /><br />DVD Extras: Bonus movie: "Zombie \'90: Extreme Pestilence"; and Trailers for other Shock-o-Rama released films'
example corresponding training labels:  0


In [5]:
vocab_size = 10000 # maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
embedding_dim = 16 # 
max_length = 120 # maximum length of of review to be kept
trunc_type='post' # trucate review from the end beyond 120 words
oov_tok = "<UNK>" # unknown  


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences) # fit on training sentences 

word_index = tokenizer.word_index
print ("word index type and length: ", type(word_index), len(word_index))


for x in list(word_index)[450:454]:
  print ("example word index key value pair: ", x, word_index[x])

sequences = tokenizer.texts_to_sequences(training_sentences)
print ("tokenizer.texts_to_sequences type: ", type(sequences))
print ("length of sequences: ", len(sequences))
print ("an example sequence: ", sequences[450])

padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
# 
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



word index type and length:  <class 'dict'> 86539
example word index key value pair:  mr 451
example word index key value pair:  overall 452
example word index key value pair:  called 453
example word index key value pair:  children 454
tokenizer.texts_to_sequences type:  <class 'list'>
length of sequences:  25000
an example sequence:  [1, 17, 2334, 1, 3, 680, 1, 703, 5270, 14, 35, 5659, 2520, 1022, 2, 54, 89, 2284, 394, 5, 2520, 1068, 108, 1, 7456, 600, 33, 2, 274, 7, 24, 226, 126, 363, 2716, 8, 8, 703, 5270, 300, 3704, 6715, 30, 323, 5, 4, 600, 742, 497, 1, 26, 1938, 1226, 7, 2, 1031, 30, 3168, 3704, 48, 61, 682, 85, 76, 2, 3512, 5, 2, 1, 3, 4, 2749, 16, 8704, 2, 1165, 15, 524, 15, 1226, 48, 879, 27, 61, 1225, 3484, 3704, 21, 4960, 1981, 3704, 1, 4, 115, 6, 522, 1226, 562, 10, 21, 24, 4, 159, 115, 10, 21, 164, 2, 1, 9, 148, 113, 5, 4, 600, 777, 1861, 1463, 16, 30, 5, 70, 1165, 8, 8, 10, 253, 41, 192, 2520, 195, 6, 157, 46, 13, 3704, 7, 2, 2778, 268, 482, 160, 600, 48, 6, 192, 273, 37

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) # reverse the key value pair in word_index dict

# compare with the previous cell 
for y in list(reverse_word_index)[450:454]:
  print ("example word index key value pair: ", y, reverse_word_index[y]) 

# def decode_review(text):
#     return ' '.join([reverse_word_index.get(i, '?') for i in text])

# print(decode_review(padded[1]))
# print(training_sentences[1])

example word index key value pair:  451 mr
example word index key value pair:  452 overall
example word index key value pair:  453 called
example word index key value pair:  454 children


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 200us/sample - loss: 0.4920 - accuracy: 0.7485 - val_loss: 0.3846 - val_accuracy: 0.8269
Epoch 2/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.2412 - accuracy: 0.9056 - val_loss: 0.3768 - val_accuracy: 0.8345
Epoch 3/10
25000/25000 [==============================] - 5s 187us/sample - loss: 0.0957 - accuracy: 0.9757 - val_loss: 0.4435 - val_accuracy: 0.8294
Epoch 4/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0247 - accuracy: 0.9969 - val_loss: 0.5189 - val_accuracy: 0.8261
Epoch 5/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0060 - accuracy: 0.9995 - val_loss: 0.5855 - val_accuracy: 0.8266
Epoch 6/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6367 - val_accuracy: 0.8268
Epoch 7/10
25000/25000 [==========================

In [9]:
e = model.layers[0] # 0th layer or the 1st layer is indeed the embedding layer
weights = e.get_weights()[0] # get weights from the embeddings layer
print(weights.shape) # shape: (vocab_size, embedding_dim) # we are considering a 16 dimensional space 

(10000, 16)


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import io

out_v = io.open('/content/gdrive/My Drive/Colab Notebooks/NLP_Week2/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('/content/gdrive/My Drive/Colab Notebooks/NLP_Week2/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:

## !!!!! Not needed !!!! 
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs.tsv')
#   files.download('meta.tsv')

In [13]:
sentence = ["The movie was awful. Don't waste your money on it."]
sequence_test = tokenizer.texts_to_sequences(sentence)
print(sequence_test)

padded_test = pad_sequences(sequence_test,maxlen=max_length, truncating=trunc_type)

ypr = model.predict(padded_test)  
print ("prediction label: ", ypr)

[[2, 18, 14, 380, 174, 446, 131, 277, 22, 10]]
prediction label:  [[6.153002e-05]]
